In [1]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import DecisionTreeRandomForests.py

Populating the interactive namespace from numpy and matplotlib


In [3]:
'''
Helpful data processing functions

'''


'''
Creates a new randomized data and labels
Input:
    replace_TF    True or False
'''
def randomizeDataMatrix(data, labels, num_samples, replace_TF):
    new_indices = np.random.choice(range(0,len(data)), size=num_samples, replace=replace_TF)
    new_data = np.zeros((num_samples, data.shape[1]))
    new_labels = np.zeros(num_samples)
    #print new_data.shape
    for sample_i in xrange(0, num_samples):
        new_data[sample_i, :] = data[new_indices[sample_i], :]
        new_labels[sample_i] = labels[new_indices[sample_i]]
    
    return new_data, new_labels


'''
Takes in an array like object and returns True if there is a '?'
'''
def isThereQuestionMark(sample_values):
    num_entries = len(sample_values)
    for feature_i in xrange(0, num_entries):
        if sample_values[feature_i] == '?':
            #print 'True'
            return True

    #print 'False'
    return False

'''
Calculates validation error
'''
def calculateValidationError(predictions, labels_valid):
    return sum([a == b for a, b in zip(predictions, labels_valid)]) / float(len(predictions))


'''
Reads a csv file and then creates the data and labels array

Getting a csv file into an array for decision trees is magical 
'''
def readCSVToBinarizedArray(filepath):


    # Read cvs into pandas dataframe
    pd_of_csv = pandas.read_csv(filepath)
    # Take each row of the dataframe and convert to a list
    list_of_sample_data_dicts = []
    list_of_sample_labels = []
    num_samples = len(pd_of_csv)
    for sample_i in xrange(0, num_samples):
        dict_of_sample = pd_of_csv.loc[sample_i].to_dict()
        if not isThereQuestionMark(dict_of_sample.values()): # Check there are no missing values, ignore those samples for now
            # Create a list of dictionaries
            try: 
                list_of_sample_labels.append(dict_of_sample.pop('label'))
            except KeyError:
                pass
            list_of_sample_data_dicts.append(dict_of_sample)    

    # Binarize
    from sklearn.feature_extraction import DictVectorizer
    dict_vectorizer = DictVectorizer(sparse=False)    

    # Create array of data
    data_array = dict_vectorizer.fit_transform(list_of_sample_data_dicts)
    #dict_vectorizer.inverse_transform(census_array)
    return data_array, np.asarray(list_of_sample_labels), dict_vectorizer

'''
Convert CSV to binarized array using an existing dict vectorizer
'''
def readCSVToBinarizedArrayUsingTransform(filepath, dict_vectorizer):


    # Read cvs into pandas dataframe
    pd_of_csv = pandas.read_csv(filepath)
    # Take each row of the dataframe and convert to a list
    list_of_sample_data_dicts = []
    num_samples = len(pd_of_csv)
    for sample_i in xrange(0, num_samples):
        dict_of_sample = pd_of_csv.loc[sample_i].to_dict()
        '''
        if not isThereQuestionMark(dict_of_sample.values()): # Check there are no missing values, ignore those samples for now
            
            # Create a list of dictionaries
            try: 
                list_of_sample_labels.append(dict_of_sample.pop('label'))
            except KeyError:
                pass
            '''
        list_of_sample_data_dicts.append(dict_vectorizer.transform(dict_of_sample))    

    #print num_samples
    # Create array of data
    #data_array = dict_vectorizer.transform(list_of_sample_dicts)
    #dict_vectorizer.inverse_transform(census_array)
    return np.squeeze(np.asarray(list_of_sample_data_dicts))

'''
Split data into training and validation sets
'''
def splitTrainValidation(data, labels, num_validation_points):
    train_data = data[0:len(data)-num_validation_points, :]
    train_labels = labels[0:len(labels)-num_validation_points]
    validation_data = data[len(data)-num_validation_points:, :]
    validation_labels = labels[len(data)-num_validation_points:]
    return train_data, train_labels, validation_data, validation_labels

# Decision Tree

In [ ]:
'''
SPAM

'''

# Load data
spam_data = scipy.io.loadmat("spam_data")
data_train = spam_data['training_data']
labels_train = spam_data['training_labels']
labels_train = labels_train.transpose().flatten()
r_data, r_labels = randomizeDataMatrix(data_train, labels_train, len(data_train), False)
data_train = r_data[0:4000]
labels_train = r_labels[0:4000]
data_validation = r_data[4000:]
labels_validation = r_labels[4000:]


In [4]:
# Create decision tree
spamDT = DecisionTreeRandomForests.DecisionTree(25)
spamDT.train(data_train, labels_train)
predictions = spamDT.predict(data_validation)
error = calculateValidationError(predictions, labels_validation)
print 'The validation accuracy for the decision tree is: %.5f' %error

The validation accuracy for the decision tree is: 0.84130


In [5]:
# Create random forest
# (num_trees, max_tree_depth, num_samples_per_tree, num_possible_features_per_split)
spamRF = DecisionTreeRandomForests.RandomForest(99, 25, 1000, 20)
spamRF.train(data_train, labels_train)
predictions = spamRF.predict(data_validation)
error = calculateValidationError(predictions, labels_validation)
print 'The validation accuracy for the random forest is: %.5f' %error

The validation accuracy for the random forest is: 0.83959


In [6]:
# Trace the path of a sample point
prediction, path = spamDT.tracePredict(r_data[25])
features = ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'perscription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumnes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'recrod', 'out', ';', '$', '#', '!', '(', '[','&']
print "For sample #25, which is classified as ham, the splits are:"
for i in xrange(0, len(path)-1):
    print "Split #", i+1, "is '",features[path[i][0][0]],"'", path[i][1], path[i][0][1]  

For sample #25, which is classified as ham, the splits are:
Split # 1 is ' meter ' < 0.323457294647
Split # 2 is ' ( ' < 0.776984139949
Split # 3 is ' volumnes ' < 0.0569105691057
Split # 4 is ' & ' < 0.141981064114
Split # 5 is ' perscription ' < 0.0216919739696
Split # 6 is ' ; ' < 0.301921266538
Split # 7 is ' pain ' < 0.0198126104283
Split # 8 is ' [ ' < 0.0268306092867
Split # 9 is ' memo ' < 0.0115176151762
Split # 10 is ' path ' < 0.0158620689655
Split # 11 is ' bank ' < 0.0276243391952
Split # 12 is ' energy ' < 0.0533799542348
Split # 13 is ' drug ' < 0.00531914893617
Split # 14 is ' differ ' < 0.00672043010753
Split # 15 is ' spam ' < 0.00679347826087
Split # 16 is ' message ' < 0.0547904556725
Split # 17 is ' private ' < 0.0124976596143
Split # 18 is ' planning ' < 0.00460829493088
Split # 19 is ' business ' < 0.0355620155039
Split # 20 is ' revision ' < 0.00314465408805
Split # 21 is ' height ' < 0.00153374233129
Split # 22 is ' featured ' < 0.00153846153846
Split # 23 is '

# Random Forest

In [7]:
# Find the root split of all trees
all_tree_splits = []
split_freq = np.zeros(32)
for i in xrange(0, len(spamRF.trees)):
    split = spamRF.trees[i].root.split_rule
# Get the data per side of the tree to determine whether < or >
    left_data, left_labels, right_data, right_labels = divideDataForChildren(r_data, r_labels, split)
    if len(left_data) > len(right_data):
        all_tree_splits.append([split, 0])
    else:
        all_tree_splits.append([split, 1])

# Count the frequency of splitting on the features and the threshold values
common_split = {}
split_freq = np.zeros(32)
for i in xrange(0, len(spamRF.trees)):
    split = all_tree_splits[i][0]
    if split[0] not in common_split:
        common_split[split[0]]=[]
    common_split[split[0]].append(split[1])
    split_freq[split[0]] += 1
    
# Sort by most common feature
split_freq_dict = dict(zip(np.arange(0, 100), split_freq))

import operator
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
#sorted_x = sorted(x.items(), key=operator.itemgetter(1))

sorted_common_split = sorted(split_freq_dict.items(), key=operator.itemgetter(1))

# Print the most common splits
print "For the random forest, the most common splits made at the root nodes of the trees are:"
for i in xrange(0, len(features)):
    if sorted_common_split[-1-i][1] > 0:
        feature = sorted_common_split[-1-i][0]
        threshold = np.mean(common_split[sorted_common_split[-1-i][0]])
        num_trees = sorted_common_split[-1-i][1]
        print "'",features[feature],"'", '< approx', threshold, "(", int(num_trees), "trees )"
        
      

For the random forest, the most common splits made at the root nodes of the trees are:
' ! ' < approx 0.955805173397 ( 37 trees )
' meter ' < approx 0.331808474386 ( 30 trees )
' volumnes ' < approx 0.112926855822 ( 13 trees )
' & ' < approx 0.21250299418 ( 10 trees )
' money ' < approx 0.110665516855 ( 4 trees )
' perscription ' < approx 0.0476169851967 ( 3 trees )
' ( ' < approx 0.903805719595 ( 1 trees )
' featured ' < approx 0.0213114754098 ( 1 trees )


# Documentation

## a) Decision Tree

The stopping criteria for the decision trees are:
- all the data at the current node belongs to one class

or

- the max depth of the tree has been reached

Splits are on single features. For each feature, the mean of the means is used as the spliting value. The feature that minimizes entropy is selected. 

The relevant classes are:
- DecisionTree
- Node

The most important functions are:
- growTree, which recursively adds nodes, either children nodes or leaf nodes
- segmentor, which determins the split rule
- impurity, which calculates the impurity of a split


## b) Random Forest

This random forest implementation has hyperparameters of the number of trees, the number of data points per tree, and the max depth of each tree. The random forest uses bootstrapping for each tree and bagging for each split. The decision trees in the random forest are modified from the standalone decision tree to call a modified segmentor. This segmentor is modified to select splits from a random subset of the features instead of across all features. 

The relevant classes are:
- RandomForest, which contains a list of RandomDecisionTrees and bootstraps to grow the RandomDecisionTrees with different data
- RandomDecisionTree (modified from DecisionTree to use growRandomizedTree)

THe most important functions are:
- growRandomizedTree (modifed from growTree to use randomizedSegmentor)
- randomizedSegmentor (modified from segmentor to choose splits from a subset of features)